In [1]:
import sys
sys.path.append("../lib")
from BinanceAPI import *
from datetime import datetime
import pandas as pd
import numpy as np
import time
import os

# Global variables

In [2]:
api_key, api_secret, api_passphrase = "", "", ""
exchange = "Binance"
symbol = "BTCUSDT"

# Load kucoin API

In [3]:
api = BinanceAPI(api_key, api_secret)

# Util function

In [4]:
def toDatetime(ts):
    return datetime.fromtimestamp(ts)

def toTimestamp(dt):
    dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
    return datetime.timestamp(dt)

# Load klines data

In [5]:
# Example responses from API documentation
# [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]

In [6]:
# get klines
etimestr = "2022-04-20 23:59:00"
etime = int(toTimestamp(etimestr)) * 1000
stime = etime - 3600 * 24 * 1000
klines = api.getKlines(symbol, "5m", stime=stime, etime=etime, limit=1000)

In [7]:
klines

[[1650384000000,
  '41445.88000000',
  '41497.09000000',
  '41411.00000000',
  '41467.27000000',
  '188.38591000',
  1650384299999,
  '7808384.35992510',
  4087,
  '99.61135000',
  '4128352.83401000',
  '0'],
 [1650384300000,
  '41467.28000000',
  '41522.00000000',
  '41457.94000000',
  '41472.09000000',
  '321.65010000',
  1650384599999,
  '13349909.34894780',
  5064,
  '100.00096000',
  '4149416.11659680',
  '0'],
 [1650384600000,
  '41472.08000000',
  '41500.00000000',
  '41462.52000000',
  '41484.37000000',
  '83.00939000',
  1650384899999,
  '3443302.72258330',
  3980,
  '42.86949000',
  '1778152.45641100',
  '0'],
 [1650384900000,
  '41484.38000000',
  '41520.54000000',
  '41425.32000000',
  '41511.96000000',
  '122.57740000',
  1650385199999,
  '5083839.65449200',
  4670,
  '47.38436000',
  '1965134.53821250',
  '0'],
 [1650385200000,
  '41511.96000000',
  '41546.63000000',
  '41490.00000000',
  '41490.00000000',
  '117.81481000',
  1650385499999,
  '4891129.58339410',
  4318,
 

In [8]:
len(klines)

288

In [9]:
toDatetime(int(klines[-1][0]/1000))

datetime.datetime(2022, 4, 20, 23, 55)

In [10]:
toDatetime(int(klines[0][0]/1000))

datetime.datetime(2022, 4, 20, 0, 0)

# Pandas dataframe

In [11]:
def computeDF(klines):
    klines = [[pd.Timestamp(int(x[0]/1000), unit='s', tz='Asia/Hong_Kong'), \
            float(x[1]), \
            float(x[2]), \
            float(x[3]), \
            float(x[4]), \
            float(x[5]), \
            float(x[6]), \
            float(x[7]), \
            float(x[8]), \
            float(x[9]), \
            float(x[10])] \
            for x in klines]
    df = pd.DataFrame(np.array(klines), \
        columns=['Timestamp', 'open', 'high', 'low', 'close', \
        'volume', 'close_time', 'quote_asset_volume', 'num_trades', \
        'take_buy_base_vol', 'take_buy_quote_vol'])
    df = df.set_index("Timestamp") 
    return df 

In [12]:
df = computeDF(klines)

In [13]:
df.head(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-20 00:00:00+08:00,41445.9,41497.1,41411,41467.3,188.386,1.65038e+12,7.80838e+06,4087,99.6114,4.12835e+06
2022-04-20 00:05:00+08:00,41467.3,41522,41457.9,41472.1,321.65,1.65038e+12,1.33499e+07,5064,100.001,4.14942e+06
2022-04-20 00:10:00+08:00,41472.1,41500,41462.5,41484.4,83.0094,1.65038e+12,3.4433e+06,3980,42.8695,1.77815e+06
2022-04-20 00:15:00+08:00,41484.4,41520.5,41425.3,41512,122.577,1.65039e+12,5.08384e+06,4670,47.3844,1.96513e+06
2022-04-20 00:20:00+08:00,41512,41546.6,41490,41490,117.815,1.65039e+12,4.89113e+06,4318,42.907,1.7813e+06
2022-04-20 00:25:00+08:00,41490,41518.2,41480,41489.4,144.157,1.65039e+12,5.98213e+06,4176,72.383,3.0036e+06
2022-04-20 00:30:00+08:00,41489.4,41518.9,41464.7,41496.6,115.339,1.65039e+12,4.78453e+06,4293,57.8785,2.40089e+06
2022-04-20 00:35:00+08:00,41496.6,41499.3,41270,41308.3,281.245,1.65039e+12,1.16294e+07,7734,118.616,4.9033e+06
2022-04-20 00:40:00+08:00,41308.3,41332.4,41236.6,41301,222.819,1.65039e+12,9.19706e+06,7395,100.602,4.1524e+06


# Save pandas dataframe

In [14]:
def saveParquet(df, path):
    df.to_parquet(path)

In [15]:
path2save = "test.parquet"
saveParquet(df, path2save)

In [21]:
!ls .

binance-kline-data-download.ipynb  test.parquet


# Load pandas dataframe from file

In [16]:
df = pd.read_parquet(path2save)

In [17]:
df.head(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-20 00:00:00+08:00,41445.88,41497.09,41411.00,41467.27,188.38591,1.650384e+12,7.808384e+06,4087.0,99.61135,4.128353e+06
2022-04-20 00:05:00+08:00,41467.28,41522.00,41457.94,41472.09,321.65010,1.650385e+12,1.334991e+07,5064.0,100.00096,4.149416e+06
2022-04-20 00:10:00+08:00,41472.08,41500.00,41462.52,41484.37,83.00939,1.650385e+12,3.443303e+06,3980.0,42.86949,1.778152e+06
2022-04-20 00:15:00+08:00,41484.38,41520.54,41425.32,41511.96,122.57740,1.650385e+12,5.083840e+06,4670.0,47.38436,1.965135e+06
2022-04-20 00:20:00+08:00,41511.96,41546.63,41490.00,41490.00,117.81481,1.650385e+12,4.891130e+06,4318.0,42.90701,1.781298e+06
2022-04-20 00:25:00+08:00,41490.01,41518.24,41480.00,41489.42,144.15723,1.650386e+12,5.982130e+06,4176.0,72.38302,3.003600e+06
2022-04-20 00:30:00+08:00,41489.43,41518.89,41464.73,41496.57,115.33942,1.650386e+12,4.784531e+06,4293.0,57.87849,2.400886e+06
2022-04-20 00:35:00+08:00,41496.58,41499.29,41270.00,41308.27,281.24468,1.650386e+12,1.162936e+07,7734.0,118.61590,4.903296e+06
2022-04-20 00:40:00+08:00,41308.28,41332.43,41236.55,41301.00,222.81879,1.650387e+12,9.197055e+06,7395.0,100.60205,4.152397e+06


In [18]:
df.tail(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-20 23:10:00+08:00,41297.05,41351.82,41257.53,41337.02,343.37671,1.650468e+12,1.417877e+07,5955.0,171.40208,7.077550e+06
2022-04-20 23:15:00+08:00,41337.01,41343.31,41085.00,41103.55,748.92534,1.650468e+12,3.086075e+07,10491.0,339.94740,1.400802e+07
2022-04-20 23:20:00+08:00,41103.54,41142.31,41054.10,41110.00,428.86600,1.650468e+12,1.762966e+07,7042.0,216.37110,8.894167e+06
2022-04-20 23:25:00+08:00,41110.01,41210.88,41101.01,41184.78,222.83431,1.650469e+12,9.169403e+06,4735.0,93.55973,3.850194e+06
2022-04-20 23:30:00+08:00,41184.79,41242.44,41139.53,41200.87,259.84402,1.650469e+12,1.070783e+07,4006.0,116.09509,4.784091e+06
2022-04-20 23:35:00+08:00,41200.88,41237.75,41162.85,41205.21,157.26885,1.650469e+12,6.481412e+06,3585.0,68.46222,2.821427e+06
2022-04-20 23:40:00+08:00,41205.21,41268.10,41202.80,41257.10,126.86915,1.650469e+12,5.230367e+06,3498.0,67.53847,2.784267e+06
2022-04-20 23:45:00+08:00,41257.10,41300.46,41251.33,41260.01,131.30517,1.650470e+12,5.419875e+06,3491.0,69.31917,2.861131e+06
2022-04-20 23:50:00+08:00,41260.02,41272.19,41206.57,41249.94,229.69284,1.650470e+12,9.470240e+06,4017.0,102.17486,4.212045e+06
